### Step 1 - Loads packages required

In [1]:
import pandas as pd
import pandasql as ps

### Step 2 - Interface file exploration to validate separators and text qualifiers

In [2]:
from custom_procedures import newestFile
sourceFolder = '/content/drive/Shareddrives/Esperanto/Supports/Suppliers/'
sourceFile = newestFile(sourceFolder)
if sourceFile:
    print(sourceFile)
    if os.path.exists(sourceFile):
        #textFile = open(sourceFile, mode="r", encoding="utf-8")
        textFile = open(sourceFile, mode="r", encoding='latin-1')
        if textFile:
            textLine = textFile.readline().strip()
            print(textLine)
            numTextLines = 0
            while textLine:
                if numTextLines <= 10:
                    textLine = textFile.readline().strip()
                    print(textLine)
                else:
                    break
                numTextLines += 1
            print(numTextLines, ' text lines read')

/content/drive/Shareddrives/Esperanto/Supports/Suppliers/1. SUPPLIERS_INTERFACE_COPPEL.csv
'SISTEMA_LEGADO';'NUMERO_PROVEEDOR';'BATCH_ID';'IMPORT_ACTION';'VENDOR_NAME';'VENDOR_NAME_NEW';'SEGMENT1';'VENDOR_NAME_ALT';'ORGANIZATION_TYPE_LOOKUP_CODE';'VENDOR_TYPE_LOOKUP_CODE';'END_DATE_ACTIVE';'BUSINESS_RELATIONSHIP';'PARENT_SUPPLIER_NAME';'ALIAS';'DUNS_NUMBER';'ONE_TIME_FLAG';'CUSTOMER_NUM';'STANDARD_INDUSTRY_CLASS';'NI_NUMBER';'CORPORATE_WEBSITE';'CHIEF_EXECUTIVE_TITLE';'CHIEF_EXECUTIVE_NAME';'BUS_CLASS_NOT_APPLICABLE';'TAX_COUNTRY_CODE';'NUM_1099';'FEDERAL_REPORTABLE_FLAG';'TYPE_1099';'STATE_REPORTABLE_FLAG';'TAX_REPORTING_NAME';'NAME_CONTROL';'TAX_VERIFICATION_DATE';'ALLOW_AWT_FLAG';'AWT_GROUP_NAME';'VAT_CODE';'VAT_REGISTRATION_NUM';'AUTO_TAX_CALC_OVERRIDE';'PAYMENT_METHOD_LOOKUP_CODE';'DELIVERY_CHANNEL';'BRANK_INSTRUCTION1';'BRANK_INSTRUCTION2';'BRANK_INSTRUCTION_DETAILS';'SETTLEMENT_PRIORITY';'PAYMENT_TEXT_MESSAGE_1';'PAYMENT_TEXT_MESSAGE_2';'PAYMENT_TEXT_MESSAGE_3';'IBY_BANK_CHARGE_

### Step 3 - Loads interface data

In [3]:
df = pd.read_csv(sourceFile, sep=';', quotechar=chr(39), encoding='latin-1')
for columnName in df.columns: 
    print(columnName)


SISTEMA_LEGADO
NUMERO_PROVEEDOR
BATCH_ID
IMPORT_ACTION
VENDOR_NAME
VENDOR_NAME_NEW
SEGMENT1
VENDOR_NAME_ALT
ORGANIZATION_TYPE_LOOKUP_CODE
VENDOR_TYPE_LOOKUP_CODE
END_DATE_ACTIVE
BUSINESS_RELATIONSHIP
PARENT_SUPPLIER_NAME
ALIAS
DUNS_NUMBER
ONE_TIME_FLAG
CUSTOMER_NUM
STANDARD_INDUSTRY_CLASS
NI_NUMBER
CORPORATE_WEBSITE
CHIEF_EXECUTIVE_TITLE
CHIEF_EXECUTIVE_NAME
BUS_CLASS_NOT_APPLICABLE
TAX_COUNTRY_CODE
NUM_1099
FEDERAL_REPORTABLE_FLAG
TYPE_1099
STATE_REPORTABLE_FLAG
TAX_REPORTING_NAME
NAME_CONTROL
TAX_VERIFICATION_DATE
ALLOW_AWT_FLAG
AWT_GROUP_NAME
VAT_CODE
VAT_REGISTRATION_NUM
AUTO_TAX_CALC_OVERRIDE
PAYMENT_METHOD_LOOKUP_CODE
DELIVERY_CHANNEL
BRANK_INSTRUCTION1
BRANK_INSTRUCTION2
BRANK_INSTRUCTION_DETAILS
SETTLEMENT_PRIORITY
PAYMENT_TEXT_MESSAGE_1
PAYMENT_TEXT_MESSAGE_2
PAYMENT_TEXT_MESSAGE_3
IBY_BANK_CHARGE_BEARER
PAYMENT_REASON_CODE
PAYMENT_REASON_COMMENTS
PAYMENT_FORMAT_CODE
ATTRIBUTE_CATEGORY
ATTRIBUTE1
ATTRIBUTE2
ATTRIBUTE3
ATTRIBUTE4
ATTRIBUTE5
ATTRIBUTE6
ATTRIBUTE7
ATTRIBUTE8
ATTR

### Step 4 - 

In [4]:
print(df.head)

<bound method NDFrame.head of        SISTEMA_LEGADO NUMERO_PROVEEDOR  BATCH_ID IMPORT_ACTION  \
0          CONSTRUNET            15239       NaN        CREATE   
1          CONSTRUNET            15795       NaN        CREATE   
2          CONSTRUNET            16109       NaN        CREATE   
3          CONSTRUNET            18077       NaN        CREATE   
4          CONSTRUNET            18785       NaN        CREATE   
...               ...              ...       ...           ...   
171544            SIM            46928       NaN        CREATE   
171545            SIM            49868       NaN        CREATE   
171546            SIM            50499       NaN        CREATE   
171547            SIM            53816       NaN        CREATE   
171548            SIM            54130       NaN        CREATE   

                                      VENDOR_NAME  VENDOR_NAME_NEW  SEGMENT1  \
0                    GRUPO SANDSTORM GAM SA DE CV              NaN       NaN   
1                

### Step 5 - Create a reduced dataset to optimize response times

In [5]:
sql_query = '''
SELECT SISTEMA_LEGADO,NUMERO_PROVEEDOR,IMPORT_ACTION,VENDOR_NAME,ORGANIZATION_TYPE_LOOKUP_CODE,
VENDOR_TYPE_LOOKUP_CODE,BUSINESS_RELATIONSHIP,TAX_COUNTRY_CODE,NUM_1099,ALLOW_AWT_FLAG,
PAYMENT_METHOD_LOOKUP_CODE
FROM df
'''
reducedDf = ps.sqldf(sql_query)

### Step 6 - Exploratory Analysis what comes from where

In [6]:
print('total number of rows in dataset:')
print(len(reducedDf.index))
print(reducedDf.columns)
sql_query1 = '''
SELECT SISTEMA_LEGADO,COUNT(*) aS NumRecords FROM reducedDf
GROUP BY SISTEMA_LEGADO
ORDER BY NumRecords DESC
'''
result1 = ps.sqldf(sql_query1)
print('Data received from each legacy system')
rowsNumberValidation = 0
for index1, row1 in result1.iterrows():
    print(row1['SISTEMA_LEGADO'], int(row1['NumRecords']))
    rowsNumberValidation = rowsNumberValidation + int(row1['NumRecords'])
print('Total number of records, ', rowsNumberValidation)

total number of rows in dataset:
171549
Index(['SISTEMA_LEGADO', 'NUMERO_PROVEEDOR', 'IMPORT_ACTION', 'VENDOR_NAME',
       'ORGANIZATION_TYPE_LOOKUP_CODE', 'VENDOR_TYPE_LOOKUP_CODE',
       'BUSINESS_RELATIONSHIP', 'TAX_COUNTRY_CODE', 'NUM_1099',
       'ALLOW_AWT_FLAG', 'PAYMENT_METHOD_LOOKUP_CODE'],
      dtype='object')
Data received from each legacy system
GASTOS 78057
OBRAS 78005
MOTOR DE SEGUIMIENTO 11226
CONSTRUNET 1905
SIM 1905
MXMENU 118
RXMENU 112
NEGOCIOS AFILIADOS 107
TECNOLOGIA 70
REFACCIONARIA 16
MARKETPLACE 11
SIE 11
ROPA 3
MUEBLES 2
PORTAL SARI 1
Total number of records,  171549


### Step 7 - Duplicate Suppliers by NUM_1099

In [7]:
sql_query2 = '''
SELECT NUM_1099,numObs FROM
(SELECT NUM_1099,COUNT(*) AS numObs FROM reducedDf
GROUP BY NUM_1099)
WHERE numObs>1
ORDER BY numObs DESC
'''
dupsDf = ps.sqldf(sql_query2)
print('Total number duplicated  records, ', len(dupsDf.index))

Total number duplicated  records,  78327


### Step 8 - Create dataset with duplicated records only

In [8]:
datasetWithDups = dupsDf.merge(reducedDf, on='NUM_1099', how='left')
print('total records on dataset with duplicates ', len(datasetWithDups.index))

total records on dataset with duplicates  171292


In [9]:
sql_query3 = '''
SELECT numObs,COUNT(*) AS BatchSize FROM dupsDf
GROUP BY numObs
ORDER BY BatchSize DESC
'''
dupsSummary = ps.sqldf(sql_query3)
print(dupsSummary)

   numObs  BatchSize
0       2      65407
1       3      11252
2       4       1634
3       5         25
4       6          6
5       7          1
6       8          1
7      10          1


In [10]:
sql_query4 = '''
SELECT NUM_1099,numObs FROM
(SELECT NUM_1099,COUNT(*) AS numObs FROM reducedDf
GROUP BY NUM_1099)
WHERE numObs>5
ORDER BY numObs DESC
'''
setToValidate = ps.sqldf(sql_query4)
print(setToValidate)
print(len(setToValidate.index))

        NUM_1099  numObs
0  SACE650728LK6      10
1   DBM000228J35       8
2  MOEC6102155J0       7
3   AMC950330JM3       6
4   GSI910222KY9       6
5   MPA850101D14       6
6   RSI021017JK5       6
7   SEI030113NX6       6
8   TQV1204251V2       6
9


In [12]:
sql_query5 = '''
SELECT B.* FROM
(SELECT NUM_1099 FROM setToValidate) AS A 
LEFT JOIN 
(SELECT * FROM df) AS B
ON
(A.NUM_1099=B.NUM_1099)
ORDER BY A.NUM_1099 ASC
'''
results5 = ps.sqldf(sql_query5)

In [16]:
outputFile = '/content/drive/Shareddrives/Esperanto/Supports/tmp/Duplicates_Sample.csv'
with open(outputFile, 'w') as textFile:
    headersList = results5.columns.to_list() 
    textLine = chr(44).join(headersList) + '\n'
    textFile.write(textLine)
    for index5, row5 in results5.iterrows():
        tmpVector = row5.to_list()
        tmpList =  ["''" if value is None else value for value in tmpVector]
        textLine = chr(44).join(tmpList) + '\n'
        textFile.write(textLine)
textFile.close()

,'','','','','',MX,MPA850101D14,'','','','','','',N,'','','','',Transferencia,'','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','',''

GASTOS,MPA850101D14,'',CREATE,MUNICIPIO DE PAPANTLA,'','','',Corporation,Supplier,'',SPEND_AUTHORIZED,'','','','','','','','','','','',MX,MPA850101D14,'','','','','','',N,'','','','',Transferencia,'','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','